In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Input
import warnings

warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
# Read data from CSV file
file_path = 'GlobalTemperatures.csv'
df = pd.read_csv(file_path)

# Impute LandAverageTemperature and LandAverageTemperatureUncertainty with mean
df['LandAverageTemperature'].fillna(df['LandAverageTemperature'].mean(), inplace=True)
df['LandAverageTemperatureUncertainty'].fillna(df['LandAverageTemperatureUncertainty'].mean(), inplace=True)

# For columns with 1200 missing values, drop those rows
cols_to_dropna = ['LandMaxTemperature', 'LandMaxTemperatureUncertainty', 'LandMinTemperature', 'LandMinTemperatureUncertainty', 'LandAndOceanAverageTemperature', 'LandAndOceanAverageTemperatureUncertainty']
df.dropna(subset=cols_to_dropna, inplace=True)

In [3]:
# Add Year and Month columns based on 'dt' column
df['Year'] = pd.to_datetime(df['dt']).dt.year
df['Month'] = pd.to_datetime(df['dt']).dt.month

# Prepare X (features) and y (target)
X = df.drop(['LandAverageTemperature', 'dt'], axis=1)
y = df['LandAverageTemperature']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale X and y using MinMaxScaler
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

# Fit and transform the training data
X_train_scaled = scaler_x.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

# Only transform the testing data
X_test_scaled = scaler_x.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

# Reshape X_train_scaled and X_test_scaled for RNN input
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Print the shapes to verify
print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)
print("y_train_scaled shape:", y_train_scaled.shape)
print("y_test_scaled shape:", y_test_scaled.shape)

X_train_scaled shape: (1593, 9, 1)
X_test_scaled shape: (399, 9, 1)
y_train_scaled shape: (1593, 1)
y_test_scaled shape: (399, 1)


In [4]:
# Define the GRU model
model = Sequential()
model.add(Input(shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))
model.add(GRU(100, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 9, 100)              │          30,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 9, 100)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 100)                 │          60,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 91,601 (357.82 KB)

 Trainable params: 91,601 (357.82 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Train the model
history = model.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=32, validation_split=0.2,verbose=2)

Epoch 1/100
40/40 - 3s - 73ms/step - loss: 0.0910 - val_loss: 0.0426
Epoch 2/100
40/40 - 0s - 8ms/step - loss: 0.0331 - val_loss: 0.0119
Epoch 3/100
40/40 - 0s - 8ms/step - loss: 0.0168 - val_loss: 0.0094
Epoch 4/100
40/40 - 0s - 7ms/step - loss: 0.0146 - val_loss: 0.0093
Epoch 5/100
40/40 - 0s - 9ms/step - loss: 0.0131 - val_loss: 0.0068
Epoch 6/100
40/40 - 0s - 9ms/step - loss: 0.0121 - val_loss: 0.0059
Epoch 7/100
40/40 - 0s - 9ms/step - loss: 0.0109 - val_loss: 0.0064
Epoch 8/100
40/40 - 0s - 9ms/step - loss: 0.0102 - val_loss: 0.0056
Epoch 9/100
40/40 - 0s - 9ms/step - loss: 0.0089 - val_loss: 0.0040
Epoch 10/100
40/40 - 0s - 8ms/step - loss: 0.0081 - val_loss: 0.0045
Epoch 11/100
40/40 - 0s - 8ms/step - loss: 0.0067 - val_loss: 0.0034
Epoch 12/100
40/40 - 0s - 8ms/step - loss: 0.0061 - val_loss: 0.0021
Epoch 13/100
40/40 - 0s - 8ms/step - loss: 0.0058 - val_loss: 0.0024
Epoch 14/100
40/40 - 0s - 8ms/step - loss: 0.0056 - val_loss: 0.0018
Epoch 15/100
40/40 - 0s - 8ms/step - loss:

In [6]:
# Make predictions
predictions = model.predict(X_test_scaled)
predictions = scaler_y.inverse_transform(predictions)

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


In [7]:
# Compare predictions with actual values
actual = scaler_y.inverse_transform(y_test_scaled)
for i in range(len(predictions)):
    print(f"Actual: {actual[i][0]}, Predicted: {predictions[i][0]}")

Actual: 3.88, Predicted: 3.7594563961029053
Actual: 8.689, Predicted: 8.816386222839355
Actual: 13.622, Predicted: 13.977884292602539
Actual: 2.335, Predicted: 2.938523054122925
Actual: 5.952999999999999, Predicted: 6.066404342651367
Actual: 4.869, Predicted: 4.831769943237305
Actual: 14.768, Predicted: 15.103588104248047
Actual: 7.423999999999999, Predicted: 7.708240509033203
Actual: 4.103, Predicted: 3.860334873199463
Actual: 4.519, Predicted: 4.503964424133301
Actual: 14.021, Predicted: 14.504230499267578
Actual: 14.034, Predicted: 14.371270179748535
Actual: 4.85, Predicted: 5.267367839813232
Actual: 9.453, Predicted: 9.792223930358887
Actual: 6.222, Predicted: 6.09636116027832
Actual: 14.445, Predicted: 14.871575355529785
Actual: 11.062, Predicted: 10.831170082092285
Actual: 2.455, Predicted: 2.8312814235687256
Actual: 6.273, Predicted: 6.277585029602051
Actual: 4.303, Predicted: 4.081684112548828
Actual: 5.066, Predicted: 5.073912620544434
Actual: 10.941999999999998, Predicted: 11

In [8]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Calculate metrics
mae = mean_absolute_error(actual, predictions)
mse = mean_squared_error(actual, predictions)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

Mean Absolute Error (MAE): 0.35085869968146616
Mean Squared Error (MSE): 0.19697589123732714
Root Mean Squared Error (RMSE): 0.4438196607151683
